In [1]:
# 저장 된 비디오를 읽고 3d pose estimation 이 된 frame을 다시 비디오로 저장.

import os
import sys

import cv2
from PIL import Image
import numpy as np
import torch
import torchvision

model_path = os.path.abspath(os.path.join('..'))
if model_path not in sys.path:
    sys.path.append(model_path)

from colorhandpose3d.model.ColorHandPose3D import ColorHandPose3D
from colorhandpose3d.model.HandNet import HandNet
from colorhandpose3d.utils.general import *
from colorhandpose3d.utils.transforms import *


from matplotlib import pyplot as plt
from tqdm import tqdm
import time

# Initialize models and load weights
weight_path = "../saved/"
use_cuda = torch.cuda.is_available()
chp3d = ColorHandPose3D(weight_path, crop_size=224)

if use_cuda:
    chp3d.cuda()




# Initialize OpenCV object
video_name = 'test_easy.avi'
cap = cv2.VideoCapture(os.path.join("../outputs/in_video/", video_name))  # Open video
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(frame_width, frame_height, length, end=",")

# Define image transforms
transform0 = torchvision.transforms.ToPILImage()
transform1 = torchvision.transforms.ToTensor()
transform2 = torchvision.transforms.Resize((224, 224))
transform3 = torchvision.transforms.Normalize(mean=[0.641, 0.619, 0.612], std=[0.254, 0.262, 0.258])
transform4 = torchvision.transforms.Resize((240, 320))
preprocess_chp3d = torchvision.transforms.Compose([transform0, transform2, transform1])
preprocess_handnet = torchvision.transforms.Compose([transform1, transform3])

def main():
    import gc
    # Preprocessing recorded frames
    frames = []
    i = 0
    while cap.isOpened():
        ret, frame = cap.read()  # frame: numpy array, (H480, W640, C3)

        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB
            sample_original = preprocess_chp3d(frame).unsqueeze(0)
#             sample_original = transform1(frame).unsqueeze(0)  # (C3, H480, W640)
            sample_chp = sample_original - 0.5
            sample_handnet = preprocess_handnet(frame)
            sample_handnet = sample_handnet.unsqueeze(0)
            hand_side = torch.tensor([[1.0, 0.0]])

#             if use_cuda:
#                 sample = sample.cuda()
#                 hand_side = hand_side.cuda()

            frames.append([sample_handnet, sample_chp, sample_original, hand_side])
            _frame = transform0(frame)
            _frame.save("../outputs/in_video/{0:04d}.png".format(i+1))
            i += 1

        
            del sample_original
            del sample_chp
            del sample_handnet
            del hand_side
            gc.collect()
            print(i)

        else:
            break

    # Pass frames through chp3d
    passFrames(frames)

    cap.release()
    cv2.destroyAllWindows()


def passFrames(frames):

    # Video writer
#     fourcc = cv2.VideoWriter_fourcc(*"DIVX")
#     out = cv2.VideoWriter("../outputs/after_test.avi", fourcc, 25.0, (640, 480))
    
#     past_keypoint = []
    for i, frame in tqdm(enumerate(frames)):
        sample_handnet = frame[0]
        sample_chp = frame[1]
        sample_original = frame[2]
        hand_side = frame[3]
        
        if use_cuda:
            sample_handnet = sample_handnet.cuda()
            sample_chp = sample_chp.cuda()
            hand_side = hand_side.cuda()
            
        
        # Pass frame through chp3d
        inputs = [sample_handnet, sample_chp]
        s = time.time()
        coords_xyz_rel_normed, keypoint_scoremap, image_crop, centers, scale_crop, hand_mask = chp3d(
            inputs, hand_side, deploy=True
        )
        print('Time for processing 1 frame: {}s'.format(time.time() - s))
        

        # hand flag
        if coords_xyz_rel_normed is None:
            img = transform4(transform0(sample_original.squeeze()))
            fig = plt.figure()
            plt.imshow(img)
            plt.axis("off")
            plt.tight_layout()
            plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
            pil = fig2img(fig)
            pil.save("../outputs/out_video/ver0_1/{0:04d}.png".format(i+1))
#             plt.savefig("../outputs/out_video/ver0_3/{0:04d}.png".format(i+1))
            plt.close()
            
            past_keypoint = []
            
        else:
            # Back to CPU
            if use_cuda is True:
                coords_xyz_rel_normed = coords_xyz_rel_normed.cpu()
                keypoint_scoremap = keypoint_scoremap.cpu()
                image_crop = image_crop.cpu()
                centers = centers.cpu()
                scale_crop = scale_crop.cpu()
                hand_mask = hand_mask.cpu()

            keypoint_coords3d = coords_xyz_rel_normed.detach().numpy()
            keypoint_coords3d = keypoint_coords3d.squeeze()
            

            keypoint_coords_crop = detect_keypoints(keypoint_scoremap[0].detach().numpy())
            keypoint_coords = transform_cropped_coords(keypoint_coords_crop, centers, scale_crop, 224)
            
#             if past_keypoint:
#                 keypoint_coords = (keypoint_coords + past_keypoint[0]) / 2.0
            
            
            img = transform0(sample_original.squeeze())
            
            fig = plt.figure()
            plt.imshow(img)
            plot_hand(keypoint_coords, plt)
            plt.axis("off")
            plt.tight_layout()
            plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
            pil = fig2img(fig)
            pil = transform4(pil)
            pil.save("../outputs/out_video/ver0_1/{0:04d}.png".format(i+1))

            plt.close()
            
#             past_keypoint = [keypoint_coords]

320.0 240.0 1250,

In [2]:
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf, bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    img = Image.open(buf)
    return img

In [ ]:
main()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


0it [00:00, ?it/s]

1244
1245
1246
1247
1248
1249
tensor([[0.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.012218952178955078
Handsegnet forward time: 0.0019812583923339844


1it [00:00,  5.09it/s]

s shape torch.Size([1, 2, 224, 224])
for loop time 0.003663778305053711
hand mask sum: 0.0
Calculate single highest scoring object time: 0.034104347229003906
Crop and resize time: 0.0008878707885742188
Posenet forward time: 0.004349470138549805
Posepriornet forward time: 0.0009033679962158203
Viewpoint forward time: 0.0009567737579345703
Get normalized 3d coordinates time: 0.0008175373077392578
Flip left handed input time: 0.00014448165893554688
ScaLe heatmaps time 9.608268737792969e-05
Time for processing 1 frame: 0.05691337585449219s
tensor([[0.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.008610963821411133
Handsegnet forward time: 0.001012563705444336
s shape torch.Size([1, 2, 224, 224])
for loop time 0.003529787063598633
hand mask sum: 0.0
Calculate single highest scoring object time: 0.0339818000793457
Crop and resize time: 0.0007781982421875
Posenet forward time: 0.0034368038177490234
Posepriornet forward time: 0.0007197856903076172
Viewpoint forwar

...

tensor([[0.0001]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.007777214050292969
Handsegnet forward time: 0.001043081283569336
s shape torch.Size([1, 2, 224, 224])
for loop time 0.002989053726196289
hand mask sum: 52.0
Calculate single highest scoring object time: 0.030100107192993164
Crop and resize time: 0.0009391307830810547
Posenet forward time: 0.0019524097442626953
Posepriornet forward time: 0.0007045269012451172
Viewpoint forward time: 0.0009322166442871094
Get normalized 3d coordinates time: 0.0006103515625
Flip left handed input time: 0.00012302398681640625
ScaLe heatmaps time 8.988380432128906e-05
Time for processing 1 frame: 0.044809818267822266s
tensor([[0.0001]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.007653951644897461
Handsegnet forward time: 0.0011355876922607422
s shape torch.Size([1, 2, 224, 224])
for loop time 0.0030145645141601562
hand mask sum: 8.0
Calculate single highest scoring object time: 0.0306024551391601

690it [01:58,  5.99it/s]

Posepriornet forward time: 0.0010154247283935547
Viewpoint forward time: 0.001016378402709961
Get normalized 3d coordinates time: 0.0006043910980224609
Flip left handed input time: 0.00012540817260742188
ScaLe heatmaps time 9.417533874511719e-05
Time for processing 1 frame: 0.0457463264465332s
tensor([[0.0001]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.007656574249267578
Handsegnet forward time: 0.0010042190551757812
s shape torch.Size([1, 2, 224, 224])
for loop time 0.0030019283294677734
hand mask sum: 1.0
Calculate single highest scoring object time: 0.030395984649658203
Crop and resize time: 0.0009372234344482422
Posenet forward time: 0.001981973648071289
Posepriornet forward time: 0.0007221698760986328
Viewpoint forward time: 0.0009329319000244141
Get normalized 3d coordinates time: 0.0005819797515869141
Flip left handed input time: 0.00011920928955078125
ScaLe heatmaps time 8.416175842285156e-05
Time for processing 1 frame: 0.04502296447753906s


692it [01:59,  6.01it/s]

tensor([[0.0001]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.007742404937744141
Handsegnet forward time: 0.0015316009521484375
s shape torch.Size([1, 2, 224, 224])
for loop time 0.0029981136322021484
hand mask sum: 2.0
Calculate single highest scoring object time: 0.02988910675048828
Crop and resize time: 0.00093841552734375
Posenet forward time: 0.0019497871398925781
Posepriornet forward time: 0.0007150173187255859
Viewpoint forward time: 0.0009379386901855469
Get normalized 3d coordinates time: 0.0005793571472167969
Flip left handed input time: 0.00012040138244628906
ScaLe heatmaps time 9.107589721679688e-05
Time for processing 1 frame: 0.04516911506652832s
tensor([[0.0001]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.00766444206237793
Handsegnet forward time: 0.0011265277862548828
s shape torch.Size([1, 2, 224, 224])
for loop time 0.0029866695404052734
hand mask sum: 8.0
Calculate single highest scoring object time: 0.03047347068786